In [1]:
import pandas as pd
import time
from tqdm.auto import tqdm
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# 크롤링 전 세팅
chrome_driver_path = '/Users/yejin/workspace/chromedriver'

# 크롤링 URL
# 향후 찾고 싶은 영상 URL만 변경
url_path = 'https://www.youtube.com/watch?v=j8eL8Xg3Xx8&t=3s'

# 크롤링 반복 횟수
# repeat 1번당 리뷰 20개정도 가져옴; repeat = 댓글 수/20?
repeat = 3

In [3]:
# 댓글 수 가져오기


In [3]:
# 댓글 작성자
commenter_lst = []
# 댓글
comment_lst=[]
# 좋아요 개수
like_count_lst = []
# 크리에이터 하트 여부
heart_exist_lst = []
# 조회수
view_count_lst = []
# 영상제목
title_lst = []
# 더보기란
more_lst = []
# 영상 업로드 날짜
upload_date_lst = []

# chrome 열기
with Chrome(executable_path = chrome_driver_path) as driver:
    # 찾으려는 대상이 불러올 때까지 지정된 시간만큼 대기하도록 설정한다.
    # 인자는 초(second) 단위이며, Default 값은 0초이다. 
    wait = WebDriverWait(driver, 20) # chrome 로딩 시간
    driver.get(url_path) # 영상 url
    time.sleep(3)
    
    # (자동실행 방지)유튜브 실행 시 자동 영상 재생일 경우, 영상 종료되면 바로 다음 영상으로 넘어가게 된다.
    # 이를 방지하기 위해, 유튜브 영상 중지 후 크롤링 진행
    if driver.find_element_by_class_name("ytp-play-button").get_attribute('aria-label') == '일시중지(k)':
        driver.find_element_by_class_name("ytp-play-button").click()
    else:
        pass
    
    # 최초 1회 PAGE_DOWN
    wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.PAGE_DOWN)
    time.sleep(3)

    # END 반복 실행
    # 실행 횟수 체크 - tqdm: 진행 상황을 Progress Bar 형태로 한눈에 확인
    for item in tqdm(range(repeat)): # END버튼 반복 횟수, 1회당 20개씩 댓글 업데이트 
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(1) # END버튼 클릭 이후, 1초 대기 후, 다시 END 버튼 진행
                
    ## 크롤링 데이터 수집 진행

    # 작성자 가져오기
    # 댓글 작성자 중 확인된 사용자, 공식 아티스트 채널 값은 text로 가져올 시, (공백) 처리됨
    try:
        for commenter in tqdm(wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#author-text')))):
            # 작성자 이름 없는 경우에, 공백 표시
            # 확인된 사용자, 공식 아티스트 채널의 경우, innertext를 가져옴
            if commenter.text != '':
                commenter_lst.append(commenter.text)
            else:
                # innerText: 얻을 수 있는 텍스트의 근사값 -> 체크 표시 있는 사용자 이름 가져옴
                commenter_temp = commenter.get_attribute("innerText").strip().replace('\n', '')
                commenter_lst.append(commenter_temp)                
    except:
        # 크롤링 값이 없을 경우에
        commenter_lst.append('')

    # 댓글 가져오기    
    try:
        for comment in tqdm(wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#content-text')))):
            if comment.text != '':
                comment_temp = comment.text.replace('\n', ' ')
                comment_lst.append(comment_temp)
            else:
                comment_lst.append(' ')            
    except:
        # 크롤링 값이 없을 경우에
        comment_lst.append('')
                
    # 좋아요 개수 가져오기
    try:
        for like_count in tqdm(wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#vote-count-middle')))):
            if like_count.text != '':
                like_count_lst.append(like_count.text)
            else:
                like_count_lst.append('0')
    except:
        # 좋아요 개수가 없을 경우에
        like_count_lst.append('0')
        
    # 크리에이터 하트 여부 체크하기
    for creater_heart in tqdm(wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#creator-heart')))):
        # 크리에이터 하트 html 존재 여부로 체크
        try:
            if creater_heart.find_element_by_css_selector('#creator-heart-button'):
                heart_exist_lst.append('하트')
            else:
                heart_exist_lst.append('없음')
        except:
            # 크롤링 값이 없을 경우에
            heart_exist_lst.append('없음')
    
    #제목 가져오기
    try:
        for title in tqdm(wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#container > h1 > yt-formatted-string')))):
            if title.text != '':
                title_lst.append(title.text)
            else:
                title_lst.append('none')
    except:
        # 제목이 없을 경우에
        title_lst.append('none')
    
    #더보기 가져오기
#     while True:
#         try:
#             more_button= driver.find_element_by_css_selector('#more > yt-formatted-string')
#             more_button.click()
#             time.sleep(1)
#             for more in tqdm(wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#description > yt-formatted-string')))):
#                 if more.text != '':
#                     more_lst.append(more.text)
#                 else:
#                     more_lst.append('none')
#         except:
#             # 더보기 없을 경우에
#             more_lst.append('none')
    
    
    # 조회수 가져오기
    try:
        for view_count in tqdm(wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#count')))):
            if view_count != '':
                view_count_lst.append(view_count.text)
            else:
                view_count_lst.append('0')
    except:
        view_count_lst.append('0')
    
    # 영상 업로드 날짜
    try:
        for upload_date in tqdm(wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#info-strings')))):
            if upload_date != '':
                upload_date_lst.append(upload_date.text)
            else:
                upload_date_lst.append('0')
    except:
        upload_date_lst.append('0')
        
print('done')

/var/folders/tz/2xhtx7px4kng48d7grkk8mwm0000gn/T/ipykernel_6269/3422998985.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with Chrome(executable_path = chrome_driver_path) as driver:
/var/folders/tz/2xhtx7px4kng48d7grkk8mwm0000gn/T/ipykernel_6269/3422998985.py:28: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  if driver.find_element_by_class_name("ytp-play-button").get_attribute('aria-label') == '일시중지(k)':
/var/folders/tz/2xhtx7px4kng48d7grkk8mwm0000gn/T/ipykernel_6269/3422998985.py:29: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name("ytp-play-button").click()


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

...

/Users/yejin/opt/anaconda3/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:484: UserWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  warnings.warn("find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead")


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

done


In [4]:
view_count = pd.DataFrame({'조회수': view_count_lst})
view_count

,조회수
0,
1,"조회수 9,103,848회"
2,"댓글 4,454개"


In [5]:
upload_date = pd.DataFrame({'업로드날짜': upload_date_lst})
upload_date

,업로드날짜
0,2021. 10. 29.


In [6]:
title = pd.DataFrame({'제목': title_lst})
title

,제목
0,진짜..눈치 1도 안보고 작정하고 약 빨고 만든 레전드 찐으른 드라마 ㅋㅋㅋ


In [ ]:
more_lst

In [7]:
# 저장 위치
#save_path = r'(파일 저장 위치)'

df = pd.DataFrame({'댓글 작성자' : commenter_lst,
                   '댓글' : comment_lst,
                   '좋아요 개수' : like_count_lst,
                   '하트 유/무': heart_exist_lst})

# 인덱스 1부터 실행
df.index = df.index+1
df

,댓글 작성자,댓글,좋아요 개수,하트 유/무
1,지무비 : G Movie,"티빙 오리지날, ""술꾼 도시 여자들"" 1화~4화까지의 소개 영상입니다. 술꾼도시...",1.5천,없음
2,김우야,한선화 캐스팅은 진짜 신의한수다... ㅋㅋㅋ 너무 찰떡같이 연기를 기가 막히게 잘함...,1만,없음
3,Jimmmy,한선화 나올때마다 웃겨 죽겠음ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,5.5천,없음
4,앨리,진짜 한선화 캐릭터 개웃곀ㅋㅋㅋㅋㅋㅋㅋ 찰떡소화야,6.3천,없음
5,강제규,"불륜, 막장, 재벌 가족, 연애사 이야기만 하던 드라마가 소재가 다양해지니 얼마나 ...",910,없음
6,김짱군,한선화 연기 개잘한닼ㅋㅋㅋㅋㅋ내친구같애........ 저런 친구가 있다는게 더 레전...,5.6천,없음
7,튼튼,14:33 여긴 아무리 봐도 너무 웃김 ㅋㅋㅋㅋㅋ 한선화 연기력 극대치 ㅋㅋㅋㅋㅋㅋ...,632,없음
8,z ommoc,덕분에 2화 막 다 봤음. 졸라 잼나네. 알콜인으로써 안볼 수 없다 ㅋㅋㅋㅋㅋㅋㅋㅋ...,2.1천,없음
9,do el,정은지 연기 왤케 잘 하냨ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 한선화는 캐릭터가 왜 이렇게 잘...,2.2천,없음
10,ᄆᄂᄋᄅ,한선화 연기 미쳤냐고 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,9.7천,없음


In [8]:
save_path = '/Users/yejin/workspace/'
df.to_csv('0323.csv')

In [9]:
t = pd.read_csv('0323.csv')
t

,Unnamed: 0,댓글 작성자,댓글,좋아요 개수,하트 유/무
0,1,지무비 : G Movie,"티빙 오리지날, ""술꾼 도시 여자들"" 1화~4화까지의 소개 영상입니다. 술꾼도시...",1.5천,없음
1,2,김우야,한선화 캐스팅은 진짜 신의한수다... ㅋㅋㅋ 너무 찰떡같이 연기를 기가 막히게 잘함...,1만,없음
2,3,Jimmmy,한선화 나올때마다 웃겨 죽겠음ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,5.5천,없음
3,4,앨리,진짜 한선화 캐릭터 개웃곀ㅋㅋㅋㅋㅋㅋㅋ 찰떡소화야,6.3천,없음
4,5,강제규,"불륜, 막장, 재벌 가족, 연애사 이야기만 하던 드라마가 소재가 다양해지니 얼마나 ...",910,없음
5,6,김짱군,한선화 연기 개잘한닼ㅋㅋㅋㅋㅋ내친구같애........ 저런 친구가 있다는게 더 레전...,5.6천,없음
6,7,튼튼,14:33 여긴 아무리 봐도 너무 웃김 ㅋㅋㅋㅋㅋ 한선화 연기력 극대치 ㅋㅋㅋㅋㅋㅋ...,632,없음
7,8,z ommoc,덕분에 2화 막 다 봤음. 졸라 잼나네. 알콜인으로써 안볼 수 없다 ㅋㅋㅋㅋㅋㅋㅋㅋ...,2.1천,없음
8,9,do el,정은지 연기 왤케 잘 하냨ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 한선화는 캐릭터가 왜 이렇게 잘...,2.2천,없음
9,10,ᄆᄂᄋᄅ,한선화 연기 미쳤냐고 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,9.7천,없음
